# Установка необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import os     

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

import scipy.stats
from scipy import stats

# Стандартное импортирование plotly
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import pydotplus
 
# Использование cufflinks в офлайн-режиме
import cufflinks as cf
cf.go_offline()
from ipywidgets import Image 
# Настройка глобальной темы cufflinks
cf.set_config_file(world_readable=True, theme='pearl', offline=True)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics
%matplotlib inline


# используем .dot формат для визуализации дерева
from ipywidgets import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

In [2]:
# #Путь к файлу
# path=r'C:\Users\kartopelka\Google Диск\Study'
# dataname='\Data.xlsx'
# #Путь к файлу тестового датасета
# path=r'C:\Users\kartopelka\Google Диск\Study'
# dataname2='\Data2.xlsx'
# #Путь к файлу prediction
# path=r'C:\Users\kartopelka\Google Диск\Study'
# datanamep='\Data2_pred.xlsx'


In [3]:
df = pd.read_excel ('./Data.xlsx', index_col=None)
df_2 = pd.read_excel ('./Data2.xlsx', index_col=None)
df_pred = pd.read_excel ('./Data2_pred.xlsx', index_col=None)


XLRDError: Excel xlsx file; not supported

In [ ]:
# df = pd.read_excel (path+dataname, index_col=None)
# df_2 = pd.read_excel (path+dataname2, index_col=None)
# df_pred = pd.read_excel (path+datanamep, index_col=None)

In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
df_pred.set_index('Шифр', inplace=True)
df_pred.head()

In [ ]:
df['Hyper'].astype('int64')
df["группа"].astype('int64')
df.set_index('Шифр', inplace=True)

In [ ]:
df

In [ ]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Get the columns with > 25% missing
missing_df = missing_values_table(df);
missing_df.head()

In [ ]:
missing_columns = list(missing_df[missing_df['% of Total Values'] > 25].index)
print('We will remove %d columns.' % len(missing_columns))

In [ ]:
missing_columns

In [ ]:
# Drop the columns
df.drop(columns = list(missing_columns),inplace=True)

In [ ]:
gya11 = df.iloc[35]
gya12 = df.iloc[36]
gya1 = df.iloc[25]

group1 = df[df['группа'] == 1] 
group2 = df[df['группа'] == 2] 
hyper = df[df['Hyper'] == 1] # Показатели группы, где произошла гиперстимуляция
group2_n = group2[group2['Hyper'] == 0] # Показатели исследуемой группы, где гиперстимуляция НЕ произошла

group1_median = group1.median() #Медианные показатели по контрольной группе
group1_mean = group1.mean() #средние показатели по контрольной группе

group2_median = group2.median()  #Медианные показатели по Исследуемой группе
group2_mean = group2.mean()  # средние показатели по Исследуемой группе

hyper_median = hyper.median() #Медианные показатели по группе с гиперстимуляцией
hyper_mean = hyper.mean() #Средние показатели по группе с гиперстимуляцией

group2_n_median = group2_n.median() #Медианные показатели по группе, где гиперстимуляция НЕ произошла
group2_n_mean = group2_n.mean() #Средние показатели по группе, где гиперстимуляция НЕ произошла

In [ ]:
# Calculate first and third quartile
g1_first_quartile = group1.quantile(0.25) 
g1_third_quartile = group1.quantile(0.75)

g2_first_quartile = group2_n.quantile(0.25) 
g2_third_quartile = group2_n.quantile(0.75)
# Interquartile range
iqr1 = g1_third_quartile - g1_first_quartile  #Межквартальынй размаж группы 1
iqr2 = g2_third_quartile - g2_first_quartile  #Межквартальынй размаж группы 2, где не произогла гиперстимуляция

In [ ]:
# Заполнение пропусков в данных медианными значениями для каждой из групп по параметрам
group1_new = group1.fillna(group1_median) 
group2_new = group2_n.fillna(group2_n_median)
hyper_new = hyper.fillna(hyper_median)

df_new = pd.concat([group1_new,group2_new,hyper_new], axis=0, sort=False)

In [ ]:
dfc_abc = pd.concat([group1_median, group2_n_median, hyper_median, group1_mean, group2_n_mean, hyper_mean], axis=1,  keys=['Контроль мед', 'Группа 2Р мед','Гипер мед', 'Контроль ср','2 группа Р ср.','Гипер ср'], ignore_index=False)

In [ ]:
hyper_des = hyper.describe()
group1_des = group1.describe()
group2_des = group2_n.describe()
dfc_des = pd.concat([group1_des, group2_des, hyper_des], axis=0)

In [ ]:
u_statistic, uVal = stats.mannwhitneyu(group1_new['Фолл'], hyper_new['Фолл'])
print ('U value:')
print (uVal)

In [ ]:
u_statistic, uVal = stats.mannwhitneyu(group2_new['Фолл'], hyper_new['Фолл'])
print ('U value:')
print (uVal)

In [ ]:
# calculate the correlation matrix
corr = df_new.corr(method='spearman')

# display the correlation matrix
# display(corr)

# plot the correlation heatmap
corr.iplot(kind='heatmap', colors='spectral',filename='./corr_spearman.html',asPlot=False)

In [ ]:
# # Create correlation matrix
# corr_matrix = df_new.corr(method='spearman').abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find index of feature columns with correlation greater than 0.90
# to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

In [ ]:
# to_drop

In [ ]:
# df_new_after = df_new.drop(df_new[to_drop], axis=1)
df_new_after = df_new

In [ ]:
df_new_after.corr(method="spearman").iplot(kind='heatmap', colorscale='spectral')

In [ ]:
group1_after = df_new_after[df_new_after['группа'] == 1]
group2_after = df_new_after[df_new_after['группа'] == 2]
group2_n_after = group2_after[group2_after['Hyper'] == 0]

group2_after = df_new_after[df_new_after['Hyper'] == 1]

hyper_after = df_new_after[df_new_after['Hyper'] == 1]

group2_AML = df_new_after[df_new_after['группа'] == 2]

In [ ]:
df_new_after.describe()

In [ ]:
dfn = df_new_after
df_new_after.drop('группа', axis = 1, inplace=True)
df_new_after.drop('Фолл', axis = 1, inplace=True)

In [ ]:
X = df_new_after.drop('Hyper', axis = 1)
y = df_new_after['Hyper']

# Деревья решений vs Метода Ближайших соседей | CrossVal / Accuracy Metric

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y,  test_size=0.3,
                                                          random_state=42, stratify=y)

In [ ]:
X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

In [ ]:
X_train

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(random_state=17, class_weight='balanced')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
%%time
knn.fit(X_train, y_train)

In [ ]:
%%time
tree.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
knn_pred = knn.predict(X_holdout)
metrics.balanced_accuracy_score(y_holdout, knn_pred)


In [ ]:
tree_pred = tree.predict(X_holdout)
metrics.balanced_accuracy_score(y_holdout, tree_pred)

In [ ]:
tree_params = {'min_samples_leaf': range(1,2)}

In [ ]:
tree_grid = GridSearchCV(tree, tree_params,
                         cv=5, n_jobs=-1,
                        verbose=True)

In [ ]:
tree_grid.fit(X_train, y_train)

In [ ]:
tree_grid.best_params_

In [ ]:
tree_grid.best_score_

In [ ]:
metrics.balanced_accuracy_score(y_holdout, tree_grid.predict(X_holdout))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])

In [ ]:
knn_params = {'knn__n_neighbors': range(3, 5)}

In [ ]:
knn_grid = GridSearchCV(knn_pipe, knn_params,
                         cv=5, n_jobs=-1,
                        verbose=True)

In [ ]:
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_params_, knn_grid.best_score_

In [ ]:
metrics.balanced_accuracy_score(y_holdout, knn_grid.predict(X_holdout))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100,  n_jobs=-1, random_state=10, class_weight='balanced', criterion='entropy', oob_score=True)
print(np.mean(cross_val_score(forest, X_train, y_train, cv=10)))

In [ ]:
# forest_params = {'min_samples_leaf': range(1, 2),}

In [ ]:
# # rf = RandomForestClassifier(n_estimators=100, ).fit(X_train, y_train)
# forest_grid = GridSearchCV(forest, forest_params,
#                          cv=10, n_jobs=-1,
#                          verbose=1)

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
# forest_grid.best_params_, forest_grid.best_score_, forest_grid.best_estimator_

In [ ]:
metrics.balanced_accuracy_score(y_holdout, forest.predict(X_holdout))

In [ ]:
forest_importances = forest.feature_importances_
forest_importances

In [ ]:
import scikitplot as skplt
skplt.estimators.plot_feature_importances(forest, title='Feature Importance', feature_names=X_holdout.columns, 
                                          max_num_features=5, 
                                          order='descending', x_tick_rotation=90,
                                          ax=None, figsize=None, title_fontsize='medium', text_fontsize='medium')

In [ ]:
# rf.predict_proba(X_holdout.loc[['ГЯ11']])

In [ ]:
est = forest

In [ ]:
# используем .dot формат для визуализации дерева
from sklearn.tree import plot_tree

# Визуализация дерева решений без Graphviz
plt.figure(figsize=(40,20))
plot_tree(tree_grid.best_estimator_, feature_names=X.columns, filled=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pylab import rc, plot
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools   

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_holdout, forest.predict(X_holdout))
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non Hyper', 'Hyper'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
sns.set(font_scale=1.5)
sns.set_color_codes("muted")

plt.figure(figsize=(10, 8))
fpr, tpr, thresholds = roc_curve(y_holdout, forest.predict_proba(X_holdout)[:,1], pos_label=1)
lw = 2
plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig("ROC.png")
plt.show()

# Random Forest |StratifiedKFold| Confusion Matrix, PPV, ROC, F1, Baloanced,

In [ ]:
X = df_new_after.drop('Hyper', axis = 1)
y = np.asarray(df_new_after["Hyper"],dtype='int8')

In [ ]:
# X = df_b.drop('Hyper', axis = 1)
# y = np.asarray(df_b["Hyper"],dtype='int8')

In [ ]:
X.shape

In [ ]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
import numpy as np
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

     

from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
# Инициализируем стратифицированную разбивку нашего датасета для валидации
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Инициализируем наш классификатор с дефолтными параметрами
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

# Обучаем на тренировочном датасете
results = cross_val_score(rfc, X, y, cv=skf)

# Оцениваем точность на тестовом датасете
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

In [ ]:
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [1, 10, 15, 20, 30, 50, 75, 100]

# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} trees".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
plt.style.use('ggplot')

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(trees_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', label='train')
ax.plot(trees_grid, test_acc.mean(axis=1), alpha=0.5, color='red', label='cv')
ax.fill_between(trees_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("N_estimators");

In [ ]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_depth_grid = [3, 5, 63]

# Обучаем на тренировочном датасете
for max_depth in max_depth_grid:
    rfc = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, oob_score=True, max_depth=max_depth)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_depth".format(max(test_acc.mean(axis=1))*100, 
                                                        max_depth_grid[np.argmax(test_acc.mean(axis=1))]))

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(max_depth_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', label='train')
ax.plot(max_depth_grid, test_acc.mean(axis=1), alpha=0.5, color='red', label='cv')
ax.fill_between(max_depth_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
ax.fill_between(max_depth_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("Max_depth");

In [ ]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
min_samples_leaf_grid = [1, 3]

# Обучаем на тренировочном датасете
for min_samples_leaf in min_samples_leaf_grid:
    rfc = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, 
                                 oob_score=True, min_samples_leaf=min_samples_leaf)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} min_samples_leaf".format(max(test_acc.mean(axis=1))*100, 
                                                        min_samples_leaf_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(min_samples_leaf_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', label='train')
ax.plot(min_samples_leaf_grid, test_acc.mean(axis=1), alpha=0.5, color='red', label='cv')
ax.fill_between(min_samples_leaf_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
ax.fill_between(min_samples_leaf_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("Min_samples_leaf");

In [ ]:
# Сделаем инициализацию параметров, по которым хотим сделать полный перебор
parameters = {'min_samples_leaf': range(1,2), 'min_samples_split': range(2,5),}
rfc = RandomForestClassifier(n_estimators=500, random_state=42, 
                             n_jobs=-1, class_weight='balanced_subsample')
gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1)
gcv.fit(X, y)

In [ ]:
gcv.best_estimator_, gcv.best_score_

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test, gcv.predict(X_test))
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Normal', 'Hyper'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
sns.set(font_scale=1.5)
sns.set_color_codes("muted")

plt.figure(figsize=(10, 8))
fpr, tpr, thresholds = roc_curve(y_test, gcv.predict_proba(X_test)[:,1], pos_label=1)
lw = 2
plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig("ROC.png")
plt.show()

In [ ]:
from sklearn import metrics
true_labels = (y_test)
predicted_labels = np.array(gcv.predict(X_test))

M = metrics.confusion_matrix(true_labels, predicted_labels)
M

### Характеристики бинарного классификатора
Основываясь на данной таблице, можно ввести несколько величин, характеризующих бинарный классификатор:
$$rec = TPR = \frac{TP}{TP + FN},\quad SPC = \frac{TN}{TN + FP},\quad prec = PPV = \frac{TP}{TP + FP},\quad FPR = 1 - SPC,$$

$$ACC = \frac{TP + TN}{TP + TN + FP + FN},\quad F1 = 2\frac{PPV\cdot TRP}{PPV + TPR}.$$

Полнота $TPR$ (True positive rate, recall, sensitivity) - доля верно классифицированных положительных примеров среди всех положительных примеров.

Специфичность $SPC$ (Specificity, true negative rate) - доля верно классифицированных отрицательных примеров среди всех отрицательных примеров.

Точность $PPV$ (Positive predictive value, precision) - доля верно классифицированных положительных примеров среди всех примеров, классифицированных положительно.

$FPR$ (False positive rate) - доля ошибочно классифицированных отрицательных примеров среди всех отрицательных примеров.

$ACC$ (Accuracy) - доля верно классифицированных примеров среди всех примеров. Основная характеристика качества классификации.

$F1$ (F1-measure) - среднее гармоническое точности и полноты. Позволяет учесть обе характеристики одновременно.

In [ ]:
PPV = metrics.precision_score(true_labels, predicted_labels)
TPR = metrics.recall_score(true_labels, predicted_labels)
F1 = metrics.f1_score(true_labels, predicted_labels)
ACC = metrics.accuracy_score(true_labels, predicted_labels)
PPV, TPR, F1, ACC

In [ ]:
# import scikitplot as skplt
# # Deriving Class probabilities
# predicted_probabilities = gcv.predict_proba(X_holdout)
# # Creating the plot
# skplt.metrics.plot_cumulative_gain(y_holdout, predicted_probabilities)
# #Scikitplot library is there to help
# skplt.metrics.plot_lift_curve(y_holdout, predicted_probabilities)

In [ ]:
from sklearn import metrics
true_labels = (y_test)
predicted_labels = np.array(gcv.predict(X_test))

M = metrics.confusion_matrix(true_labels, predicted_labels)
M

In [ ]:
PPV = metrics.precision_score(true_labels, predicted_labels)
TPR = metrics.recall_score(true_labels, predicted_labels)
F1 = metrics.f1_score(true_labels, predicted_labels)
ACC = metrics.accuracy_score(true_labels, predicted_labels)
Balanced = metrics.balanced_accuracy_score(true_labels, predicted_labels)
recall = metrics.recall_score(true_labels, predicted_labels)
PPV, TPR, F1, ACC

In [ ]:
Balanced

In [ ]:
recall

In [ ]:
# true = df_pred['Hyper']
df_pred.drop(columns = list(missing_columns),inplace=True)
df_pred = df_pred.fillna(hyper_median)
# df_pred = df_pred.drop('Hyper', axis = 1)
df_pred = df_pred.drop('Фолл', axis = 1)
# df_pred.drop('группа', axis = 1, inplace=True)

In [ ]:
hyper_median.shape , df_pred.shape

In [ ]:
import scikitplot as skplt
skplt.estimators.plot_feature_importances(gcv.best_estimator_, title='Feature Importance', feature_names=X_test.columns, 
                                          max_num_features=10, 
                                          order='descending', x_tick_rotation=90,
                                          ax=None, figsize=None, title_fontsize='medium', text_fontsize='medium')

In [ ]:
pred_lab = gcv.predict_proba(df_pred)

In [ ]:
y_probas = gcv.predict_proba(X_test)
skplt.metrics.plot_precision_recall(y_test, y_probas)

plt.show()

In [ ]:
y_probas = gcv.predict_proba(X_test)
skplt.metrics.plot_roc(y_test, y_probas)

plt.show()

In [ ]:
import scikitplot as skplt
# Deriving Class probabilities
predicted_probabilities = gcv.predict_proba(X_test)
# Creating the plot
skplt.metrics.plot_cumulative_gain(y_test, predicted_probabilities)
#Scikitplot library is there to help
skplt.metrics.plot_lift_curve(y_test, predicted_probabilities)

In [ ]:
pred_lab

In [ ]:
y_pred = (clf.predict_proba(X_holdout)[:,1] >= 0.3).astype(bool)